# Stratified K-fold cross-validation

- Note that it is possible to get test accuracy higher than validation accuracy.
- This might ring an alarm (i.e. too few test data as pointed out in https://stats.stackexchange.com/a/59632).
- However, keep in mind that when using the test data, `gs.fit()` already fit to the entire training set so the amount of data that is used to train the final model is different from the amount of data used to train during grid search. 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.datasets import load_breast_cancer

In [2]:
# Load the breast cancer dataset
dataObj = load_breast_cancer()
print(dataObj.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [3]:
# Create a dataframe
df = pd.DataFrame(dataObj.data, columns=dataObj.feature_names)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

### Setting up


In [4]:
X = dataObj.data
y = dataObj.target

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=0.30,
    stratify=y,
    random_state=1)

# Standardization
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Classifier
lr = LogisticRegression(random_state=1)

### Cross validation


In [5]:
from sklearn.model_selection import cross_validate

scores = cross_validate(estimator=lr,
                         X=X_train_std,
                         y=y_train,
                         cv=5,
                         scoring=['accuracy','f1'],
                         n_jobs=-1)

In [6]:
df = pd.DataFrame(scores)
display(df)

,fit_time,score_time,test_accuracy,test_f1
0,0.002003,0.002040,0.975000,0.980392
1,0.003998,0.001553,1.000000,1.000000
2,0.003001,0.000996,0.962500,0.970297
3,0.002503,0.002006,0.974684,0.980392
4,0.001553,0.002007,1.000000,1.000000


In [7]:
df.describe().T.loc[['test_accuracy', 'test_f1'],['mean','std']]

,mean,std
test_accuracy,0.982437,0.016806
test_f1,0.986216,0.013241


### Parameter turning (manual)


In [8]:
Cs = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

datas = []
for C in Cs:
    lr = LogisticRegression(random_state=1, C=C)
    scores = cross_validate(
        estimator=lr, X=X_train_std, y=y_train, cv=5, scoring=["accuracy", "f1"], n_jobs=-1
    )

    data = {
        "C": C,
        "accuracy_mean": scores["test_accuracy"].mean(),
        "accuracy_std": scores["test_accuracy"].std(),
        "f1_mean": scores["test_f1"].mean(),
        "f1_std": scores["test_f1"].std(),
    }
    datas.append(data)

df = pd.DataFrame.from_dict(datas)
df = df.sort_values(by="accuracy_mean", ascending=False)
display(df)

,C,accuracy_mean,accuracy_std,f1_mean,f1_std
3,1.000,0.982437,0.015032,0.986216,0.011843
2,0.100,0.974937,0.023717,0.980544,0.018421
4,10.000,0.964842,0.009314,0.972111,0.007464
5,100.000,0.952342,0.024141,0.961646,0.019942
1,0.010,0.947310,0.016461,0.959632,0.012577
6,1000.000,0.942278,0.023063,0.953320,0.019268
0,0.001,0.866930,0.026539,0.904191,0.017266


### Refit

In [9]:
# Choose the best C
C_best = df.iloc[0]["C"]
print(C)

1000.0


In [10]:
lr = LogisticRegression(random_state=1, C=C_best, max_iter=5000) # max_iter=5000 to avoid convergence warning
lr.fit(X_train_std, y_train)

LogisticRegression(max_iter=5000, random_state=1)

### Actual test result

Note that the test accuracy can be higher than CV accuracy due to different numbers of training and test data.

In [11]:
y_pred = lr.predict(X_test_std)

print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(f"F1: {f1_score(y_test, y_pred):.2f}")

Accuracy: 0.96
F1: 0.97
